In [ ]:
import os
import wave
import time
import threading
import tkinter as tk
import pyaudio
from tkinter import messagebox
import socket

class VoiceRecorder:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Voice Recorder")
        self.root.geometry("400x300")
        self.root.resizable(False, False)

        self.recording = False
        self.frames = []

        # UI Elements
        self.label = tk.Label(text="00:00:00", font=("Arial", 24))
        self.label.pack(pady=20)

        self.record_button = tk.Button(text="🎙️ Record", font=("Arial", 20, "bold"),
                                       command=self.click_handler)
        self.record_button.pack(pady=10)

        self.stop_button = tk.Button(text="🛑 Stop", font=("Arial", 20, "bold"),
                                     command=self.stop_recording, state=tk.DISABLED)
        self.stop_button.pack(pady=10)

        self.filename_entry = tk.Entry(self.root, font=("Arial", 14))
        self.filename_entry.pack(pady=10)

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

        # Start UDP server
        threading.Thread(target=self.udp_server).start()

        self.root.mainloop()

    def click_handler(self):
        if self.recording:
            messagebox.showwarning("Warning", "Recording is already in progress!")
        else:
            self.recording = True
            self.frames = []
            self.record_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.record_button.config(fg="red")
            threading.Thread(target=self.record).start()

    def stop_recording(self):
        if self.recording:
            self.recording = False
            self.record_button.config(state=tk.NORMAL)
            self.stop_button.config(state=tk.DISABLED)
            self.record_button.config(fg="black")

    def record(self):
        audio = pyaudio.PyAudio()
        stream = audio.open(format=pyaudio.paInt16, channels=1, rate=44100,
                            input=True, frames_per_buffer=1024)

        start = time.time()

        while self.recording:
            data = stream.read(1024)
            self.frames.append(data)

            passed = time.time() - start
            secs = passed % 60
            mins = (passed // 60) % 60
            hours = passed // 3600
            self.label.config(text=f"{int(hours):02d}:{int(mins):02d}:{int(secs):02d}")

        stream.stop_stream()
        stream.close()
        audio.terminate()

        self.save_recording()

    def save_recording(self):
        filename = self.filename_entry.get().strip()
        if not filename:
            filename = "recording"

        i = 1
        while os.path.exists(f"{filename}{i}.wav"):
            i += 1

        sound_file = wave.open(f"{filename}{i}.wav", "wb")
        sound_file.setnchannels(1)
        sound_file.setsampwidth(pyaudio.PyAudio().get_sample_size(pyaudio.paInt16))
        sound_file.setframerate(44100)
        sound_file.writeframes(b"".join(self.frames))
        sound_file.close()

        messagebox.showinfo("Recording Saved", f"Recording saved as {filename}{i}.wav")

    def udp_server(self):
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        server_address = ('0.0.0.0', 12345)  # Listen on all network interfaces
        sock.bind(server_address)

        while True:
            data, address = sock.recvfrom(4096)
            if data.decode() == 'start':
                self.click_handler()
            elif data.decode() == 'stop':
                self.stop_recording()

    def on_closing(self):
        if self.recording:
            self.stop_recording()
        self.root.destroy()

if __name__ == "__main__":
    VoiceRecorder()
